REFS:

> https://python-twitter.readthedocs.io/en/latest/index.html

> https://developer.twitter.com/en/docs

Info:

1st corona case in The Netherlands: 27/02/2020

1st corona case in Brazil: 26/02/2020


**TODO**

- Check functions to see if I have all I need.
- Generalize as much as possible
- Order infor per date (created_at?)
- Should I have a function also for search?
- Retrieve info for:
    - jairbolsonaro
    - picnic
    - JumboSupermarkt
    - albertheijn
    - evanescence
    - WTofficial
    
- Include in class `TweetMiner` a function to user `GetSearch`

Using [`python-twitter`](https://python-twitter.readthedocs.io/en/latest/getting_started.html) since I've been facing some time out problems with [`Tweepy`](http://docs.tweepy.org/en/latest/getting_started.html).

In [1]:
# !pip install python-twitter

# Load packages

In [33]:
# config contains Twitter credentials
import config
import twitter
import datetime
import pandas as pd
import time
TodaysDate = time.strftime("%Y-%m-%d-%H-%M")



# Setting up twitter authentication

In [34]:
consumer_key = config.consumer_key
consumer_secret = config.consumer_secret
access_token = config.access_token
access_token_secret = config.access_token_secret

api = twitter.Api(
    consumer_key         =   consumer_key,
    consumer_secret      =   consumer_secret,
    access_token_key     =   access_token,
    access_token_secret  =   access_token_secret,
    tweet_mode = 'extended'
)

# Functions

Function `mine_user_tweets` made use of [Get user_timeline](https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline)


In [115]:
import datetime
#TweetMiner function from Mike Roman

# some modification added by me - 15/04/19

class TweetMiner(object):

    
    def __init__(self, api, result_limit = 20, max_pages = 40):
        """result_limit = count that can take max 200"""
        
        self.api = api        
        self.result_limit = result_limit
        self.max_pages = max_pages
        

    def mine_user_tweets(self, user, mine_retweets=False):
        """ Mine tweets of user = screen_name or user_id"""

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= self.max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1, 
                                                        include_rts=mine_retweets)
                statuses = [ _.AsDict() for _ in statuses]
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, include_rts=mine_retweets)
                statuses = [_.AsDict() for _ in statuses]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   item['retweet_count'],
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                        'favorite_count':  item['favorite_count'],
                        'tweet_id_str':    item['id_str'],
                        'source':          item['source'],
                        
                    }
                    
            
                
                except:
                        mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   0,
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
#                         'favorite_count':  item['favorite_count'],
                        'tweet_id_str':    item['id_str'],
                        'source':          item['source'],
                    }
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
        return data

In [139]:
def processing_and_saving(df, file_name):
    
    TodaysDate = time.strftime("%Y-%m-%d-%H-%M")

    
    # Create columns 'year', 'month', 'day', 'hour', 'min' from 'created_at'
    df['created_at'] = pd.to_datetime(df['created_at'])
        
    df['year'] = df['created_at'].dt.year 
    df['month'] = df['created_at'].dt.month 
    df['day'] = df['created_at'].dt.day 
    df['hour'] = df['created_at'].dt.hour 
    df['minute'] = df['created_at'].dt.minute
    df['day_of_week'] = df['created_at'].dt.weekday
    
    df.sort_values(by='created_at',inplace = True)
    
    df = df[['mined_at', 'handle','tweet_id', 'tweet_id_str','created_at', 
             'year', 'month', 'day','day_of_week', 'hour', 'minute','retweet_count', 'source', 'text']]
    
    df.to_csv("./data_tweets/"+file_name+"_"+TodaysDate+".csv", index = False)
    
    return df
    
    

# Getting twitter by user

## Instantiate the class
---

Make sure you pass the keys dictionary and the api as arguments.

**Check:** call the object's `mine_user_tweets()` method, providing a user to pull the tweets of.

In [130]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=20, max_pages = 200)

In [131]:
jairbolsonaro = miner.mine_user_tweets(user="jairbolsonaro")

In [132]:
for x in range(5):
    print(jairbolsonaro[x]['text'])
    print('--')

- Nesta manhã, conversei com o Presidente da Rússia, Vladimir Putin.

- Concordamos em aprofundar ainda mais a cooperação entre nossos países, inclusive no combate à covid-19.

- Tratamos também dos resultados que queremos atingir na próxima Cúpula do BRICS, em São Petersburgo. https://t.co/LL08lqexsH
--
- Todos podem fazer pelo Brasil.
- Deus abençoe esta grande Nação. https://t.co/25VOz3alMx
--
i3- Divulgação de dados sobre auxílio emergencial no portal da transparência; 

i4- 680 agências da @Caixa abertas neste sábado para pagamento do auxílio emergencial. O cadastro agora pode ser feito também pelos correios.
--
i1- A verdade sobre a grande operação do auxílio emergencial. @MinCidadania @onyxlorenzoni . https://t.co/TqqvsKwxcn

i2- @govbr recupera quase R$ 30 milhões em fraudes e erros do auxílio emergencial. Operação em andamento. https://t.co/AtzDGtLPDQ https://t.co/x5S2C9a1O5
--
H- Apoio para incentivar pequenos produtores orgânicos no Brasil durante a covid-19. @Mapa_Brasil @T

## Convert the tweet ouputs to a pandas DataFrame
### jairbolsonaro

In [133]:
jairbolsonaro

[{'tweet_id': 1272529594772570112,
  'handle': 'jairbolsonaro',
  'retweet_count': 4350,
  'text': '- Nesta manhã, conversei com o Presidente da Rússia, Vladimir Putin.\n\n- Concordamos em aprofundar ainda mais a cooperação entre nossos países, inclusive no combate à covid-19.\n\n- Tratamos também dos resultados que queremos atingir na próxima Cúpula do BRICS, em São Petersburgo. https://t.co/LL08lqexsH',
  'mined_at': datetime.datetime(2020, 6, 16, 21, 5, 16, 272594),
  'created_at': 'Mon Jun 15 14:01:12 +0000 2020',
  'favorite_count': 34805,
  'tweet_id_str': '1272529594772570112',
  'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>'},
 {'tweet_id': 1272244004416098306,
  'handle': 'jairbolsonaro',
  'retweet_count': 7598,
  'text': '- Todos podem fazer pelo Brasil.\n- Deus abençoe esta grande Nação. https://t.co/25VOz3alMx',
  'mined_at': datetime.datetime(2020, 6, 16, 21, 5, 16, 272594),
  'created_at': 'Sun Jun 14 19:06:22 +0000 2020',


In [134]:
df_bolsonaro = processing_and_saving(pd.DataFrame(jairbolsonaro), 'jairbolsonaro')

In [135]:
df_bolsonaro.head()

,mined_at,handle,tweet_id,tweet_id_str,created_at,year,month,day,day_of_week,hour,minute,retweet_count,source,text
0,2020-06-16 21:05:16.272594,jairbolsonaro,1272529594772570112,1272529594772570112,2020-06-15 14:01:12+00:00,2020,6,15,0,14,1,4350,"<a href=""http://twitter.com/download/iphone"" r...","- Nesta manhã, conversei com o Presidente da R..."
1,2020-06-16 21:05:16.272594,jairbolsonaro,1272244004416098306,1272244004416098306,2020-06-14 19:06:22+00:00,2020,6,14,6,19,6,7598,"<a href=""http://twitter.com/download/iphone"" r...",- Todos podem fazer pelo Brasil.\n- Deus abenç...
2,2020-06-16 21:05:16.272594,jairbolsonaro,1272175567119540225,1272175567119540225,2020-06-14 14:34:25+00:00,2020,6,14,6,14,34,1555,"<a href=""http://twitter.com/download/iphone"" r...",i3- Divulgação de dados sobre auxílio emergenc...
3,2020-06-16 21:05:16.272594,jairbolsonaro,1272175358918496256,1272175358918496256,2020-06-14 14:33:36+00:00,2020,6,14,6,14,33,1807,"<a href=""http://twitter.com/download/iphone"" r...",i1- A verdade sobre a grande operação do auxíl...
4,2020-06-16 21:05:16.272594,jairbolsonaro,1272174672344485889,1272174672344485889,2020-06-14 14:30:52+00:00,2020,6,14,6,14,30,1052,"<a href=""http://twitter.com/download/iphone"" r...",H- Apoio para incentivar pequenos produtores o...


In [136]:
df_bolsonaro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3002 entries, 0 to 3001
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   mined_at       3002 non-null   datetime64[ns]     
 1   handle         3002 non-null   object             
 2   tweet_id       3002 non-null   int64              
 3   tweet_id_str   3002 non-null   object             
 4   created_at     3002 non-null   datetime64[ns, UTC]
 5   year           3002 non-null   int64              
 6   month          3002 non-null   int64              
 7   day            3002 non-null   int64              
 8   day_of_week    3002 non-null   int64              
 9   hour           3002 non-null   int64              
 10  minute         3002 non-null   int64              
 11  retweet_count  3002 non-null   int64              
 12  source         3002 non-null   object             
 13  text           3002 non-null   object           

In [137]:
min(df_bolsonaro.created_at),max(df_bolsonaro.created_at)

(Timestamp('2019-04-07 19:23:44+0000', tz='UTC'),
 Timestamp('2020-06-15 14:01:12+0000', tz='UTC'))

In [138]:
df_bolsonaro.sort_values(by='created_at')

,mined_at,handle,tweet_id,tweet_id_str,created_at,year,month,day,day_of_week,hour,minute,retweet_count,source,text
3001,2020-06-16 21:05:52.327931,jairbolsonaro,1114972042330292225,1114972042330292225,2019-04-07 19:23:44+00:00,2019,4,7,6,19,23,10437,"<a href=""http://twitter.com/download/iphone"" r...","Depois de o Datafolha publicar que ""Lula e Dil..."
3000,2020-06-16 21:05:52.327931,jairbolsonaro,1115008829941534731,1115008829941534731,2019-04-07 21:49:55+00:00,2019,4,7,6,21,49,2741,"<a href=""http://twitter.com/download/iphone"" r...",👍🏻 https://t.co/iLu8YVjv9e
2999,2020-06-16 21:05:52.327931,jairbolsonaro,1115185430901788672,1115185430901788672,2019-04-08 09:31:40+00:00,2019,4,8,0,9,31,6566,"<a href=""http://twitter.com/download/iphone"" r...",O alinhamento do Governo Bolsonaro com Israel ...
2998,2020-06-16 21:05:52.327931,jairbolsonaro,1115185528369029120,1115185528369029120,2019-04-08 09:32:03+00:00,2019,4,8,0,9,32,4459,"<a href=""http://twitter.com/download/iphone"" r...",A necessidade de países - mesmo que não ligado...
2997,2020-06-16 21:05:52.327931,jairbolsonaro,1115261421321883648,1115261421321883648,2019-04-08 14:33:38+00:00,2019,4,8,0,14,33,8539,"<a href=""http://twitter.com/download/iphone"" r...",Comunico a todos a indicação do Professor Abra...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2020-06-16 21:05:16.272594,jairbolsonaro,1272174672344485889,1272174672344485889,2020-06-14 14:30:52+00:00,2020,6,14,6,14,30,1052,"<a href=""http://twitter.com/download/iphone"" r...",H- Apoio para incentivar pequenos produtores o...
3,2020-06-16 21:05:16.272594,jairbolsonaro,1272175358918496256,1272175358918496256,2020-06-14 14:33:36+00:00,2020,6,14,6,14,33,1807,"<a href=""http://twitter.com/download/iphone"" r...",i1- A verdade sobre a grande operação do auxíl...
2,2020-06-16 21:05:16.272594,jairbolsonaro,1272175567119540225,1272175567119540225,2020-06-14 14:34:25+00:00,2020,6,14,6,14,34,1555,"<a href=""http://twitter.com/download/iphone"" r...",i3- Divulgação de dados sobre auxílio emergenc...
1,2020-06-16 21:05:16.272594,jairbolsonaro,1272244004416098306,1272244004416098306,2020-06-14 19:06:22+00:00,2020,6,14,6,19,6,7598,"<a href=""http://twitter.com/download/iphone"" r...",- Todos podem fazer pelo Brasil.\n- Deus abenç...


### picnic

In [142]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=20, max_pages = 100)
picnic = miner.mine_user_tweets(user="picnic")
df_picnic = processing_and_saving(pd.DataFrame(picnic), "picnic")


In [143]:
df_picnic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1995 entries, 1994 to 0
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   mined_at       1995 non-null   datetime64[ns]     
 1   handle         1995 non-null   object             
 2   tweet_id       1995 non-null   int64              
 3   tweet_id_str   1995 non-null   object             
 4   created_at     1995 non-null   datetime64[ns, UTC]
 5   year           1995 non-null   int64              
 6   month          1995 non-null   int64              
 7   day            1995 non-null   int64              
 8   day_of_week    1995 non-null   int64              
 9   hour           1995 non-null   int64              
 10  minute         1995 non-null   int64              
 11  retweet_count  1995 non-null   int64              
 12  source         1995 non-null   object             
 13  text           1995 non-null   object           

In [144]:
min(df_picnic.created_at),max(df_picnic.created_at)

(Timestamp('2019-07-11 18:11:48+0000', tz='UTC'),
 Timestamp('2020-06-16 17:33:54+0000', tz='UTC'))

### JumboSupermarkt

In [165]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=20, max_pages = 200)
JumboSupermarkt = miner.mine_user_tweets(user="JumboSupermarkt")
df_JumboSupermarkt = processing_and_saving(pd.DataFrame(JumboSupermarkt), "JumboSupermarkt")


In [166]:
df_JumboSupermarkt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3217 entries, 3216 to 0
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   mined_at       3217 non-null   datetime64[ns]     
 1   handle         3217 non-null   object             
 2   tweet_id       3217 non-null   int64              
 3   tweet_id_str   3217 non-null   object             
 4   created_at     3217 non-null   datetime64[ns, UTC]
 5   year           3217 non-null   int64              
 6   month          3217 non-null   int64              
 7   day            3217 non-null   int64              
 8   day_of_week    3217 non-null   int64              
 9   hour           3217 non-null   int64              
 10  minute         3217 non-null   int64              
 11  retweet_count  3217 non-null   int64              
 12  source         3217 non-null   object             
 13  text           3217 non-null   object           

In [167]:
min(df_JumboSupermarkt.created_at),max(df_JumboSupermarkt.created_at)

(Timestamp('2020-03-04 14:45:25+0000', tz='UTC'),
 Timestamp('2020-06-16 18:29:48+0000', tz='UTC'))

### albertheijn

In [184]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=20, max_pages = 300)
albertheijn = miner.mine_user_tweets(user="albertheijn")
df_albertheijn = processing_and_saving(pd.DataFrame(albertheijn), "albertheijn")


In [185]:
df_albertheijn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3243 entries, 3242 to 0
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   mined_at       3243 non-null   datetime64[ns]     
 1   handle         3243 non-null   object             
 2   tweet_id       3243 non-null   int64              
 3   tweet_id_str   3243 non-null   object             
 4   created_at     3243 non-null   datetime64[ns, UTC]
 5   year           3243 non-null   int64              
 6   month          3243 non-null   int64              
 7   day            3243 non-null   int64              
 8   day_of_week    3243 non-null   int64              
 9   hour           3243 non-null   int64              
 10  minute         3243 non-null   int64              
 11  retweet_count  3243 non-null   int64              
 12  source         3243 non-null   object             
 13  text           3243 non-null   object           

In [186]:
min(df_albertheijn.created_at),max(df_albertheijn.created_at)

(Timestamp('2020-03-30 08:04:02+0000', tz='UTC'),
 Timestamp('2020-06-16 16:36:50+0000', tz='UTC'))

### evanescence

In [178]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=20, max_pages = 300)
evanescence = miner.mine_user_tweets(user="evanescence")
df_evanescence = processing_and_saving(pd.DataFrame(evanescence), "evanescence")


In [179]:
df_evanescence.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1079 entries, 1078 to 0
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   mined_at       1079 non-null   datetime64[ns]     
 1   handle         1079 non-null   object             
 2   tweet_id       1079 non-null   int64              
 3   tweet_id_str   1079 non-null   object             
 4   created_at     1079 non-null   datetime64[ns, UTC]
 5   year           1079 non-null   int64              
 6   month          1079 non-null   int64              
 7   day            1079 non-null   int64              
 8   day_of_week    1079 non-null   int64              
 9   hour           1079 non-null   int64              
 10  minute         1079 non-null   int64              
 11  retweet_count  1079 non-null   int64              
 12  source         1079 non-null   object             
 13  text           1079 non-null   object           

In [180]:
min(df_evanescence.created_at),max(df_evanescence.created_at)

(Timestamp('2009-09-22 22:11:50+0000', tz='UTC'),
 Timestamp('2020-06-11 21:09:13+0000', tz='UTC'))

In [187]:
df_evanescence.tail()

,mined_at,handle,tweet_id,tweet_id_str,created_at,year,month,day,day_of_week,hour,minute,retweet_count,source,text
4,2020-06-16 22:11:36.226067,evanescence,1262801790010974208,1262801790010974208,2020-05-19 17:46:23+00:00,2020,5,19,1,17,46,138,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Amy will do an Instagram live interview tomorr...
3,2020-06-16 22:11:36.226067,evanescence,1263181934806446081,1263181934806446081,2020-05-20 18:56:56+00:00,2020,5,20,2,18,56,211,"<a href=""http://twitter.com/download/iphone"" r...",On a Mon we lost our minds &amp; started a Tik...
2,2020-06-16 22:11:36.226067,evanescence,1263908407624364033,1263908407624364033,2020-05-22 19:03:41+00:00,2020,5,22,4,19,3,94,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Happy Memorial Day Weekend! We’re offering 20%...
1,2020-06-16 22:11:36.226067,evanescence,1270744546134163457,1270744546134163457,2020-06-10 15:48:03+00:00,2020,6,10,2,15,48,322,"<a href=""http://twitter.com/download/iphone"" r...",Thanks for the inspiration! ❤️ https://t.co/ZH...
0,2020-06-16 22:11:36.226067,evanescence,1271187756794822656,1271187756794822656,2020-06-11 21:09:13+00:00,2020,6,11,3,21,9,199,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Check out the new cover we did for @MagentaMus...


### Within Temptation - WTofficial

In [181]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=20, max_pages = 300)
WTofficial = miner.mine_user_tweets(user="WTofficial")
df_WTofficial = processing_and_saving(pd.DataFrame(WTofficial), "WTofficial")


In [182]:
df_WTofficial.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3065 entries, 3064 to 0
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   mined_at       3065 non-null   datetime64[ns]     
 1   handle         3065 non-null   object             
 2   tweet_id       3065 non-null   int64              
 3   tweet_id_str   3065 non-null   object             
 4   created_at     3065 non-null   datetime64[ns, UTC]
 5   year           3065 non-null   int64              
 6   month          3065 non-null   int64              
 7   day            3065 non-null   int64              
 8   day_of_week    3065 non-null   int64              
 9   hour           3065 non-null   int64              
 10  minute         3065 non-null   int64              
 11  retweet_count  3065 non-null   int64              
 12  source         3065 non-null   object             
 13  text           3065 non-null   object           

In [183]:
min(df_WTofficial.created_at),max(df_WTofficial.created_at)

(Timestamp('2018-11-23 08:13:57+0000', tz='UTC'),
 Timestamp('2020-06-16 16:31:43+0000', tz='UTC'))

# Experimenting with GetSearch

In [189]:
def search_tweets(max_pages = 20, count = 20, raw_query = None, result_type = 'mixed'):
        """ Search tweets """

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses = api.GetSearch(raw_query=raw_query, count = count, result_type=result_type, max_id=last_tweet_id - 1)
                statuses = [ _.AsDict() for _ in statuses]
            else:
                statuses = api.GetSearch(raw_query=raw_query, count = count, result_type=result_type)
                statuses = [_.AsDict() for _ in statuses]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   item['retweet_count'],
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                        'favorite_count':  item['favorite_count'],
                        'tweet_id_str':    item['id_str'],
                        'source':          item['source'],
                        
                    }
                    
            
                
                except:
                        mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   0,
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
#                         'favorite_count':  item['favorite_count'],
                        'tweet_id_str':    item['id_str'],
                        'source':          item['source'],
                    }
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
        return data

In [198]:
raw_query="q=jairbolsonaro%2C%20corona%2C%20covid&src=typed_query"
results = search_tweets(max_pages = 15, count = 20, raw_query = raw_query, result_type = 'mixed')

In [199]:
len(results)

225

In [202]:
results[0]

{'tweet_id': 1272954468154433537,
 'handle': 'Bot_Corona_V',
 'retweet_count': 0,
 'text': 'RT @TatoDeMacedo1: Hoje, 16 de junho de 2020:\nFaz 3 meses da primeira morte pela "gripezinha" de @jairbolsonaro.\n O Brasil,epicentro mundia…',
 'mined_at': datetime.datetime(2020, 6, 16, 22, 35, 57, 942091),
 'created_at': 'Tue Jun 16 18:09:30 +0000 2020',
 'tweet_id_str': '1272954468154433537',
 'source': '<a href="https://github.com/snowlew" rel="nofollow">Alpha-sun</a>'}

In [208]:
df_search_bolsonaro_corona = processing_and_saving(pd.DataFrame(results),"search_bolsonaro_corona")

In [210]:
df_search_bolsonaro_corona.tail()

,mined_at,handle,tweet_id,tweet_id_str,created_at,year,month,day,day_of_week,hour,minute,retweet_count,source,text
196,2020-06-16 22:36:04.708898,Bot_Corona_V,1272954468091539457,1272954468091539457,2020-06-16 18:09:30+00:00,2020,6,16,1,18,9,0,"<a href=""https://github.com/snowlew"" rel=""nofo...","RT @TatoDeMacedo1: Hoje, 16 de junho de 2020:\..."
136,2020-06-16 22:36:02.236812,Bot_Corona_V,1272954468091539457,1272954468091539457,2020-06-16 18:09:30+00:00,2020,6,16,1,18,9,0,"<a href=""https://github.com/snowlew"" rel=""nofo...","RT @TatoDeMacedo1: Hoje, 16 de junho de 2020:\..."
135,2020-06-16 22:36:02.236812,Bot_Corona_V,1272954468154433537,1272954468154433537,2020-06-16 18:09:30+00:00,2020,6,16,1,18,9,0,"<a href=""https://github.com/snowlew"" rel=""nofo...","RT @TatoDeMacedo1: Hoje, 16 de junho de 2020:\..."
91,2020-06-16 22:36:01.009173,Bot_Corona_V,1272954468091539457,1272954468091539457,2020-06-16 18:09:30+00:00,2020,6,16,1,18,9,0,"<a href=""https://github.com/snowlew"" rel=""nofo...","RT @TatoDeMacedo1: Hoje, 16 de junho de 2020:\..."
0,2020-06-16 22:35:57.942091,Bot_Corona_V,1272954468154433537,1272954468154433537,2020-06-16 18:09:30+00:00,2020,6,16,1,18,9,0,"<a href=""https://github.com/snowlew"" rel=""nofo...","RT @TatoDeMacedo1: Hoje, 16 de junho de 2020:\..."


In [205]:
df_search_bolsonaro_corona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   tweet_id        225 non-null    int64         
 1   handle          225 non-null    object        
 2   retweet_count   225 non-null    int64         
 3   text            225 non-null    object        
 4   mined_at        225 non-null    datetime64[ns]
 5   created_at      225 non-null    object        
 6   tweet_id_str    225 non-null    object        
 7   source          225 non-null    object        
 8   favorite_count  15 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 15.9+ KB


In [224]:
results = api.GetSearch(raw_query="q=picnic", count = 100, result_type='popular')

In [226]:
results

[Status(ID=1272995287947030528, ScreenName=yung__xin, Created=Tue Jun 16 20:51:42 +0000 2020, Text='RT @louisvuittyong_: the fully laid picnic cloth... the neat little plates stacked with different foods 😫😫😓😭😭'),
 Status(ID=1272995282624507904, ScreenName=pvchkoo, Created=Tue Jun 16 20:51:41 +0000 2020, Text='RT @hobiesjk: 🐿 TOP 20 HOPEKOOK MOMENTS 🐰\n\n#4 — 78.4%\n\nhopekook picnic dates \nhttps://t.co/ttfoex7Mvz'),
 Status(ID=1272995279642525696, ScreenName=snaykbayt, Created=Tue Jun 16 20:51:40 +0000 2020, Text='RT @ULTRAGLOSS: me n the girlies having a socially distant summer picnic https://t.co/I4agC94XQU'),
 Status(ID=1272995274366091264, ScreenName=davetucker01, Created=Tue Jun 16 20:51:39 +0000 2020, Text='@marcusgauntlett Time it right and it’d make a nice picnic'),
 Status(ID=1272995272625438720, ScreenName=anamca_, Created=Tue Jun 16 20:51:38 +0000 2020, Text='me apetece mucho hacer un picnic'),
 Status(ID=1272995270939140096, ScreenName=bobbieshop1953, Created=Tue Jun 16 20

In [233]:
raw_query = "q=picnic%20jumbo%20ah%20delivery&src=typed_query"

picnic_ah_jumbo_search = search_tweets(max_pages = 1, count = 5, raw_query = raw_query, result_type = 'mixed')
picnic_ah_jumbo_search

[]

In [231]:
df_picnic_ah_jumbo_search = processing_and_saving(pd.DataFrame(picnic_ah_jumbo_search),"picnic_ah_jumbo_search")

KeyError: 'created_at'

In [229]:
df_picnic_ah_jumbo_search

,mined_at,handle,tweet_id,tweet_id_str,created_at,year,month,day,day_of_week,hour,minute,retweet_count,source,text
74,2020-06-16 23:00:46.229831,Geovanninhaf,1272319279891972099,1272319279891972099,2020-06-15 00:05:29+00:00,2020,6,15,0,0,5,0,"<a href=""http://twitter.com/download/iphone"" r...",Graças ao nosso querido presidente @jairbolson...
134,2020-06-16 23:00:47.884337,Geovanninhaf,1272319279891972099,1272319279891972099,2020-06-15 00:05:29+00:00,2020,6,15,0,0,5,0,"<a href=""http://twitter.com/download/iphone"" r...",Graças ao nosso querido presidente @jairbolson...
119,2020-06-16 23:00:47.571151,Geovanninhaf,1272319279891972099,1272319279891972099,2020-06-15 00:05:29+00:00,2020,6,15,0,0,5,0,"<a href=""http://twitter.com/download/iphone"" r...",Graças ao nosso querido presidente @jairbolson...
104,2020-06-16 23:00:47.235808,Geovanninhaf,1272319279891972099,1272319279891972099,2020-06-15 00:05:29+00:00,2020,6,15,0,0,5,0,"<a href=""http://twitter.com/download/iphone"" r...",Graças ao nosso querido presidente @jairbolson...
89,2020-06-16 23:00:46.641989,Geovanninhaf,1272319279891972099,1272319279891972099,2020-06-15 00:05:29+00:00,2020,6,15,0,0,5,0,"<a href=""http://twitter.com/download/iphone"" r...",Graças ao nosso querido presidente @jairbolson...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2020-06-16 23:00:47.235808,Bot_Corona_V,1272954468091539457,1272954468091539457,2020-06-16 18:09:30+00:00,2020,6,16,1,18,9,0,"<a href=""https://github.com/snowlew"" rel=""nofo...","RT @TatoDeMacedo1: Hoje, 16 de junho de 2020:\..."
106,2020-06-16 23:00:47.571151,Bot_Corona_V,1272954468091539457,1272954468091539457,2020-06-16 18:09:30+00:00,2020,6,16,1,18,9,0,"<a href=""https://github.com/snowlew"" rel=""nofo...","RT @TatoDeMacedo1: Hoje, 16 de junho de 2020:\..."
105,2020-06-16 23:00:47.571151,Bot_Corona_V,1272954468154433537,1272954468154433537,2020-06-16 18:09:30+00:00,2020,6,16,1,18,9,0,"<a href=""https://github.com/snowlew"" rel=""nofo...","RT @TatoDeMacedo1: Hoje, 16 de junho de 2020:\..."
135,2020-06-16 23:00:48.188245,Bot_Corona_V,1272954468154433537,1272954468154433537,2020-06-16 18:09:30+00:00,2020,6,16,1,18,9,0,"<a href=""https://github.com/snowlew"" rel=""nofo...","RT @TatoDeMacedo1: Hoje, 16 de junho de 2020:\..."
